In [0]:
from __future__ import division
from __future__ import print_function

import time
import math
import numpy as np
import pickle as pkl
import networkx as nx
import scipy.sparse as sp
from sklearn import preprocessing
from scipy.sparse.linalg.eigen.arpack import eigsh
import sys
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.parameter import Parameter
from torch.nn.modules.module import Module
import torch.optim as optim

In [0]:
def parse_index_file(filename):
    index = []
    for line in open(filename):
        index.append(int(line.strip()))
    return index
    
def normalize_features(mx):
    rowsum = np.array(mx.sum(1))
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    mx = r_mat_inv.dot(mx)
    return mx

def preprocess_adj(adj):
    adj = sp.coo_matrix(adj)
    rowsum = np.array(adj.sum(1))
    d_inv_sqrt = np.power(rowsum, -0.5).flatten()
    d_inv_sqrt[np.isinf(d_inv_sqrt)] = 0.
    d_mat_inv_sqrt = sp.diags(d_inv_sqrt)
    return adj.dot(d_mat_inv_sqrt).transpose().dot(d_mat_inv_sqrt).tocoo()

def sparse_mx_to_torch_sparse_tensor(sparse_mx):
    sparse_mx = sparse_mx.tocoo().astype(np.float32)
    indices = torch.from_numpy(
        np.vstack((sparse_mx.row, sparse_mx.col)).astype(np.int64))
    values = torch.from_numpy(sparse_mx.data)
    shape = torch.Size(sparse_mx.shape)
    return torch.sparse.FloatTensor(indices, values, shape)

In [0]:
def load_data(dataset_str):
  names = ['x', 'y', 'tx', 'ty', 'allx', 'ally', 'graph']
  objects = []
  for i in range(len(names)):
      with open("data/ind.{}.{}".format(dataset_str, names[i]), 'rb') as f:
              objects.append(pkl.load(f, encoding='latin1'))
  x, y, tx, ty, allx, ally, graph = tuple(objects)
  test_idx_reorder = parse_index_file("data/ind.{}.test.index".format(dataset_str))
  test_idx_range = np.sort(test_idx_reorder)
  features = sp.vstack((allx, tx)).tolil()
  features[test_idx_reorder, :] = features[test_idx_range, :]
  adj = nx.adjacency_matrix(nx.from_dict_of_lists(graph))
  labels = np.vstack((ally, ty))
  labels[test_idx_reorder, :] = labels[test_idx_range, :]
  idx_test = test_idx_range.tolist()
  idx_train = range(len(y))
  idx_val = range(len(y), len(y)+500)
  features = normalize_features(features)
  adj = preprocess_adj(adj + sp.eye(adj.shape[0]))
  features = torch.FloatTensor(np.array(features.todense()))
  labels = torch.LongTensor(np.where(labels)[1])
  adj = sparse_mx_to_torch_sparse_tensor(adj)
  idx_train = torch.LongTensor(idx_train)
  idx_val = torch.LongTensor(idx_val)
  idx_test = torch.LongTensor(idx_test)
  return adj, features, labels, idx_train, idx_val, idx_test




In [0]:
dataset="pubmed" 

In [0]:
adj, features, labels, idx_train, idx_val, idx_test = load_data(dataset)

In [0]:
class GCN_Layer(Module):

    def __init__(self, inputs, outputs, bias=True):
        super(GCN_Layer, self).__init__()
        self.inputs = input
        self.outputs = outputs
        self.weights = Parameter(torch.FloatTensor(inputs, outputs))
        if bias:
            self.bias = Parameter(torch.FloatTensor(outputs))
        else:
            self.register_parameter('bias', None)
        self.initialize_parameters()

    def initialize_parameters(self):
        sigma = 1/ math.sqrt(self.weights.size(1))
        self.weights.data.uniform_(-sigma, sigma)
        if self.bias is not None:
            self.bias.data.uniform_(-sigma, sigma)

    def forward(self, input, adj):
        xw= torch.mm(input, self.weights)
        output = torch.spmm(adj,xw)
        if self.bias is not None:
            return output + self.bias
        else:
            return output

    def __repr__(self):
        return self.__class__.__name__ + '\n Inputs = \n' + str(self.inputs) + '\n to \n' + 'Ouputs = \n' + str(self.outputs)

class GCN(nn.Module):
    def __init__(self, no_feat, no_hid, no_class, dropout):
        super(GCN, self).__init__()

        self.l1 = GCN_Layer(no_feat, no_hid)
        self.l2 = GCN_Layer(no_hid, no_class)
        self.dropout = dropout

    def forward(self, input, adj):
        out1 = F.relu(self.l1(input, adj))
        in2 = F.dropout(out1, self.dropout, training=self.training)
        out2 = self.l2(in2, adj)
        output = F.log_softmax(out2, dim=1)
        return output

In [0]:
def accuracy(output, labels):
    preds = output.max(1)[1].type_as(labels)
    correct = preds.eq(labels).double()
    correct = correct.sum()
    return correct / len(labels)
    
def train(epoch,early,minm):
    t = time.time()
    model.train()
    optimizer.zero_grad()
    output = model(features, adj)
    loss_train = F.nll_loss(output[idx_train], labels[idx_train])
    acc_train = accuracy(output[idx_train], labels[idx_train])
    loss_train.backward()
    optimizer.step()
    loss_val = F.nll_loss(output[idx_val], labels[idx_val])
    
    if loss_val<minm:
      minm=loss_val
      early=0
    else:
      early=early+1
    acc_val = accuracy(output[idx_val], labels[idx_val])
    print('Epoch: {:04d}'.format(epoch+1),
          'loss_train: {:.4f}'.format(loss_train.item()),
          'acc_train: {:.4f}'.format(acc_train.item()),
          'loss_val: {:.4f}'.format(loss_val.item()),
          'acc_val: {:.4f}'.format(acc_val.item()),
          'time: {:.4f}s'.format(time.time() - t))
    return early,minm


def test():
    model.eval()
    output = model(features, adj)
    loss_test = F.nll_loss(output[idx_test], labels[idx_test])
    acc_test = accuracy(output[idx_test], labels[idx_test])
    print("Test set results:",
          "loss= {:.4f}".format(loss_test.item()),
          "accuracy= {:.4f}".format(acc_test.item()))
    

In [0]:
model = GCN(no_feat=features.shape[1],no_hid=16,no_class=labels.max().item() + 1,dropout=0.5)
optimizer = optim.Adam(model.parameters(),lr=0.05, weight_decay=5e-4)

In [19]:
t_start = time.time()
early=0;
minm=1000;
for epoch in range(200):
    early,minm = train(epoch,early,minm)
    if early is 10:
      break;
print("Training Comlete! \nTotal time elapsed: {:.4f}s".format(time.time() - t_start))

Epoch: 0001 loss_train: 1.1122 acc_train: 0.3333 loss_val: 1.0848 acc_val: 0.3880 time: 0.0440s
Epoch: 0002 loss_train: 1.0705 acc_train: 0.4500 loss_val: 1.0769 acc_val: 0.4280 time: 0.0378s
Epoch: 0003 loss_train: 1.0445 acc_train: 0.5833 loss_val: 1.0770 acc_val: 0.4500 time: 0.0363s
Epoch: 0004 loss_train: 1.0177 acc_train: 0.5667 loss_val: 1.0771 acc_val: 0.4160 time: 0.0354s
Epoch: 0005 loss_train: 1.0235 acc_train: 0.5167 loss_val: 1.0641 acc_val: 0.4560 time: 0.0354s
Epoch: 0006 loss_train: 0.9539 acc_train: 0.6500 loss_val: 1.0286 acc_val: 0.5220 time: 0.0347s
Epoch: 0007 loss_train: 0.9209 acc_train: 0.6833 loss_val: 1.0124 acc_val: 0.5360 time: 0.0392s
Epoch: 0008 loss_train: 0.8814 acc_train: 0.7167 loss_val: 0.9765 acc_val: 0.6080 time: 0.0382s
Epoch: 0009 loss_train: 0.8447 acc_train: 0.7500 loss_val: 0.9586 acc_val: 0.6280 time: 0.0388s
Epoch: 0010 loss_train: 0.8230 acc_train: 0.8167 loss_val: 0.9479 acc_val: 0.5960 time: 0.0427s
Epoch: 0011 loss_train: 0.7603 acc_train

In [20]:
test()

Test set results: loss= 0.5885 accuracy= 0.7860
